# Imports

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime
import wmfdata
import pandas as pd
import requests
from wmfdata import hive,spark
from wmfdata.utils import print_err, pd_display_all
from wmfdata.utils import check_kerberos_auth, ensure_list

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
# METRICS_MONTH_TEXT = '2019-07'
# MEDIAWIKI_HISTORY_SNAPSHOT = '2019-07'

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)


METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")

MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

# Preparation

In [3]:

# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    "metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [4]:
date_params

{'mediawiki_history_snapshot': '2022-03',
 'metrics_month': '2022-03',
 'metrics_month_start': '2022-03-01 00:00:00',
 'metrics_month_first_day': '2022-03-01',
 'metrics_month_end': '2022-04-01 00:00:00',
 'metrics_month_last_day': '2022-03-31',
 'api_metrics_month_first_day': '20220301',
 'api_metrics_month_day_after': '20220401',
 'metrics_prev_month': '2022-02',
 'retention_cohort': '2022-01'}

# MariaDB and Hive query metrics

In [5]:
queries = {
    "active_editors": {
        "file": "queries/active_editors.sql",
        "engine": "hive"
    },
    "edits": {
        "file": "queries/edits.hql",
        "engine": "hive"
    },
    "new_editor_retention": {
        "file": "queries/new_editor_retention.hql",
         "engine": "hive"
    },
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_edits_editors": {
        "file": "queries/mobile-heavy_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_new_editor_retention": {
        "file": "queries/mobile-heavy_new_editor_retention.hql",
        "engine": "hive"
    },
    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running active_editors on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running edits on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running new_editor_retention on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_edits_editors on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running mobile-heavy_edits_editors on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running mobile-heavy_new_editor_retention on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_new_editor_retention on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Content metrics via API

In [6]:
NEW_PAGES_API = (
    # Replaces "https://wikimedia.org/api/rest_v1/metrics/" due to https://phabricator.wikimedia.org/P8605
    #"http://aqs1004.eqiad.wmnet:7232/analytics.wikimedia.org/v1/" 
    "https://wikimedia.org/api/rest_v1/metrics/"
    "edited-pages/new/{project}/all-editor-types/{page_type}/monthly/{start}/{end}"
)

headers = {
    "User-Agent": "https://github.com/wikimedia-research/Editing-movement-metrics (bot)"
}

# Create container for results
api_results = []

def get_new_pages(
    project="all-projects",
    page_type="content",
    start= date_params["api_metrics_month_first_day"],
    end= date_params["api_metrics_month_day_after"]
):
    url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )
    
    r = requests.get(url, headers=headers)
    data = r.json()["items"][0]["results"]
    frame = pd.DataFrame(data)
    frame["timestamp"] = pd.to_datetime(frame["timestamp"]).dt.tz_localize(None)
    frame = frame.rename(columns={"timestamp": "month"})
    
    return frame

In [7]:
project="all-projects",
page_type="content"
start=date_params["api_metrics_month_first_day"]
end=date_params["api_metrics_month_day_after"]

In [8]:
 url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )

## Total

In [9]:
total_new = get_new_pages().rename(columns={"new_pages": "net_new_content_pages"})
api_results.append(total_new)

## Wikidata

In [10]:
new_wd = get_new_pages(
    project="wikidata.org"
).rename(columns={
    "new_pages": "net_new_Wikidata_entities"
})
api_results.append(new_wd)

## Commons

In [11]:
new_commons = get_new_pages(
    project="commons.wikimedia.org"
).rename(columns={
    "new_pages": "net_new_Commons_content_pages"
})
api_results.append(new_commons)

## Wikipedias

In [12]:
# Get a list of project URLs (each one in a 1-tuple)
wp_domains = spark.run("""
select domain_name
from canonical_data.wikis
where database_group = "wikipedia"
""", format="raw")

# Query the API for each project and append records to a list
results = []
n = len(wp_domains)

/home/iflorez/.conda/envs/2021-08-13T18.21.10_iflorez/lib/python3.7/site-packages/wmfdata/spark.py:306: FutureWarning: The 'raw' format is deprecated. It will be removed in the next major release.
  category=FutureWarning
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [13]:

for idx, val in enumerate(wp_domains):
    domain = val[0]
    
    if idx % 10 == 0:
        msg = "Now on the {}th project of {} ({})"
        print_err(msg.format(idx, n, domain))
        
    frame = get_new_pages(project=domain).reset_index()
    frame["project"] = domain
    records = frame.to_dict("records")
    results.extend(records)
    
    # Sleep 20 milliseconds
    time.sleep(0.02)

# Turn the big list of records into a data frame
new_per_wp = pd.DataFrame(results)

# Sum across projects to get new Wikipedia articles per month
new_wp = new_per_wp.groupby("month").agg(
    {"new_pages": "sum"}
).rename(columns={"new_pages": "net_new_Wikipedia_articles"}).reset_index()

api_results.append(new_wp)

Now on the 0th project of 324 (aa.wikipedia.org)
Now on the 10th project of 324 (ang.wikipedia.org)
Now on the 20th project of 324 (av.wikipedia.org)
Now on the 30th project of 324 (be-tarask.wikipedia.org)
Now on the 40th project of 324 (br.wikipedia.org)
Now on the 50th project of 324 (chr.wikipedia.org)
Now on the 60th project of 324 (cv.wikipedia.org)
Now on the 70th project of 324 (ee.wikipedia.org)
Now on the 80th project of 324 (ff.wikipedia.org)
Now on the 90th project of 324 (gag.wikipedia.org)
Now on the 100th project of 324 (got.wikipedia.org)
Now on the 110th project of 324 (hr.wikipedia.org)
Now on the 120th project of 324 (ig.wikipedia.org)
Now on the 130th project of 324 (ja.wikipedia.org)
Now on the 140th project of 324 (kj.wikipedia.org)
Now on the 150th project of 324 (ks.wikipedia.org)
Now on the 160th project of 324 (lfn.wikipedia.org)
Now on the 170th project of 324 (lt.wikipedia.org)
Now on the 180th project of 324 (mi.wikipedia.org)
Now on the 190th project of 32

In [14]:
# Strip timezones returned by API so our month columns merge nicely
for df in api_results:
    df["month"] = df["month"].dt.tz_localize(None)

# Combining and saving metrics

In [15]:
queries["edits"]["result"] = queries["edits"]["result"].rename({"data_edits": "wikidata_edits"}, axis=1)

In [16]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]
results.extend(api_results)

In [17]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,active_editors,anonymous_edits,global_south_active_editors,global_south_edits,global_south_new_editor_retention,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,mobile_edits,net_new_Commons_content_pages,net_new_Wikidata_entities,net_new_Wikipedia_articles,net_new_content_pages,new_active_editors,new_editor_retention,non_anonymous_edits,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits
month,,,,,,,,,,,,,,,,,,,,,,,,
2021-11-01,89995.0,2018200.0,23066.0,3303526.0,NaN,2550849.0,5107.0,973932.0,0.056051,629947.0,1821699.0,543471.0,420731.0,288246.0,1460354.0,16586.0,0.080796,37416732.0,5973130.0,73409.0,0.054219,39436215.0,586487.0,16358067.0
2021-12-01,89356.0,1995954.0,23172.0,3312507.0,NaN,2672058.0,5315.0,1060115.0,0.048249,659813.0,1893692.0,696066.0,446094.0,281101.0,1653680.0,16041.0,0.072860,39301991.0,8267140.0,73315.0,0.053034,41299016.0,746046.0,16768078.0
2022-01-01,94400.0,2196791.0,24440.0,4380346.0,NaN,2692378.0,5588.0,2068915.0,0.049787,721691.0,2097066.0,703587.0,378116.0,331589.0,1670762.0,17748.0,0.062472,44069031.0,11540342.0,76652.0,0.054356,46267521.0,781837.0,17365714.0
2022-02-01,88282.0,1958744.0,22935.0,3737443.0,NaN,2381297.0,5256.0,1374497.0,0.048724,572935.0,1878139.0,569124.0,224029.0,218825.0,1248681.0,16198.0,0.063009,36620342.0,10177703.0,72084.0,0.059112,38580288.0,612714.0,13450324.0
2022-03-01,91776.0,2094120.0,23773.0,3205072.0,NaN,2426786.0,5334.0,993853.0,0.054394,582447.0,1991180.0,678076.0,356670.0,208310.0,1594924.0,17776.0,0.077532,49914361.0,10696650.0,74000.0,0.047048,52020405.0,717167.0,22334725.0


In [18]:
metrics

,active_editors,anonymous_edits,global_south_active_editors,global_south_edits,global_south_new_editor_retention,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,...,net_new_content_pages,new_active_editors,new_editor_retention,non_anonymous_edits,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits
month,,,,,,,,,,,,,,,,,,,,,
2001-01-01,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10696.0,6.0,NaN,NaN,225.0,0.0,0.014760,13783.0,0.0,0.0
2001-02-01,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,434.0,4.0,NaN,NaN,600.0,5.0,0.001362,2178.0,0.0,0.0
2001-03-01,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1309.0,12.0,0.500000,NaN,1018.0,7.0,0.000818,4078.0,0.0,0.0
2001-04-01,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1604.0,5.0,0.600000,NaN,759.0,13.0,0.003322,2735.0,0.0,0.0
2001-05-01,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3796.0,6.0,0.538462,NaN,1698.0,15.0,0.001532,5955.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-01,89995.0,2018200.0,23066.0,3303526.0,NaN,2550849.0,5107.0,973932.0,0.056051,629947.0,...,1460354.0,16586.0,0.080796,37416732.0,5973130.0,73409.0,0.054219,39436215.0,586487.0,16358067.0
2021-12-01,89356.0,1995954.0,23172.0,3312507.0,NaN,2672058.0,5315.0,1060115.0,0.048249,659813.0,...,1653680.0,16041.0,0.072860,39301991.0,8267140.0,73315.0,0.053034,41299016.0,746046.0,16768078.0
2022-01-01,94400.0,2196791.0,24440.0,4380346.0,NaN,2692378.0,5588.0,2068915.0,0.049787,721691.0,...,1670762.0,17748.0,0.062472,44069031.0,11540342.0,76652.0,0.054356,46267521.0,781837.0,17365714.0


In [19]:
metrics.to_csv(FILENAME, sep="\t")